In [ ]:
import numpy as np 
import json 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from scipy import sparse 
import pickle 

In [ ]:
train_path ='./data/train_processed.csv'
val_path =  './data/val_processed.csv'
test_path = './data/test_processed.csv'

train = pd.read_csv(train_path)
val = pd.read_csv(val_path)
test = pd.read_csv(test_path)

In [ ]:
train.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,review_id,user_id,business_id,stars,date,text,useful,...,year,month,day,language,cleaned_text,funny_label,cool_label,useful_label,funny_sampled_flag,cool_sampled_flag
0,0,0,0,O4RZMP8IFyJTNfRp0QXEsw,u0LXt3Uea_GidxRW1xcsfg,vcxvQyAggPqxcHwvJXvjGg,4,2017-01-04,Love this place!\n\nThe cakes are delicious bu...,0.0,...,2017.0,1.0,4.0,en,Love place cake delicious really rich cute cup...,0,0,0,0,0
1,1,1,1,66KqTwiQ1oB9-aTsoEN35Q,u0LXt3Uea_GidxRW1xcsfg,DKiRDPtQ5cTN-eX1oEgA9w,3,2017-01-04,It's a pub... nice and clean one.\n\nCame here...,0.0,...,2017.0,1.0,4.0,en,pub nice clean one Came grab food girl since T...,0,0,0,0,0
2,2,2,2,3KQ3_xutS5R3mX7HsPWpgw,u0LXt3Uea_GidxRW1xcsfg,zkU-WMio8g6dpRJ2Y2xqvQ,4,2017-01-04,Came here for dim sum - it's one of those mark...,0.0,...,2017.0,1.0,4.0,en,Came dim sum one mark sheet order dim sum pret...,0,0,0,0,0
3,3,3,3,p659sFGjbL8chCRZPPyFmg,u0LXt3Uea_GidxRW1xcsfg,3RlylOY452bA8rwliPUeUQ,4,2017-01-04,My sister loves this place! Probably comes by ...,1.0,...,2017.0,1.0,4.0,en,sister love place Probably come week every two...,0,0,1,1,1
4,4,4,4,1XEm85LhDeiUHdG0R_rXSQ,u0LXt3Uea_GidxRW1xcsfg,sTV4qyjDkY5cLvx2omtLyw,4,2017-02-07,Nice addition to the Distillery district! Stum...,1.0,...,2017.0,2.0,7.0,en,Nice addition Distillery district Stumbled upo...,0,0,1,0,0


In [ ]:
# train_use = train
train_fun = train[train.funny_sampled_flag==1]
train_cool = train[train.cool_sampled_flag==1]

In [ ]:
val.head()

,Unnamed: 0,Unnamed: 0.1,review_id,user_id,business_id,stars,date,text,useful,funny,cool,year,month,day,language,cleaned_text,funny_label,cool_label,useful_label
0,0,0,Ebggx4Zlc4VWReJMG1nT6w,u0LXt3Uea_GidxRW1xcsfg,l1_S1mfGbEMxfT1f9omhEA,1,2017-10-16,Terrible service and not so great drinks.\n\nW...,0.0,0.0,0.0,2017.0,10.0,16.0,en,Terrible service great drink happen plaza saw ...,0,0,0
1,1,1,jREsaout3cuhKbROVDXUFg,u0LXt3Uea_GidxRW1xcsfg,I44P6Pfoey2pArOhhx2RnA,4,2017-10-17,Cute little hole in the wall place. Two entran...,0.0,0.0,0.0,2017.0,10.0,17.0,en,Cute little hole wall place Two entrance one s...,0,0,0
2,2,2,S5BSUs-iDrwn6o5968LmEg,u0LXt3Uea_GidxRW1xcsfg,xNNAfZJkLZlAeS-I7-QwgA,3,2017-10-16,We were here on a holiday Monday where nothing...,0.0,0.0,0.0,2017.0,10.0,16.0,en,holiday Monday nothing much wa open grab takeo...,0,0,0
3,3,3,qOkKKBgIvpg3xzxJitCoCQ,tL2pS5UOmN6aAOi3Z-qFGg,Trbt0Ex85yvwT8DHoEFCvg,1,2017-10-11,Ordered food to go from here.\n\nThey send you...,2.0,1.0,0.0,2017.0,10.0,11.0,en,Ordered food go send email telling food ready ...,1,0,1
4,4,4,heSUfficKTzHOWrlIYcZ0g,4DvHNL2g_H3onCpV2zjegQ,hGfgR4eN8T2IVvviJaZdBA,5,2017-10-15,HP wanted $300 to replace a 500GB hard drive. ...,0.0,0.0,0.0,2017.0,10.0,15.0,en,HP wanted replace hard drive wa referred Santo...,0,0,0


In [ ]:
for i in [train, train_fun, train_cool, val, test]:
    i.dropna(inplace=True)
    print(i.shape)

(896945, 22)


/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


(294305, 22)
(376157, 22)
(98119, 19)
(126864, 19)


In [ ]:
train.reset_index(inplace=True)
train_fun.reset_index(inplace=True)
train_cool.reset_index(inplace=True)
val.reset_index(inplace=True)
test.reset_index(inplace=True)

In [ ]:
from sklearn import tree, ensemble
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, mean_squared_error
from xgboost import XGBClassifier

# useful

In [ ]:
model = 'doc2vec'
with open(f'./text_processing/useful_processed_text/useful_{model}_train.npy', 'rb') as f:
    train_array = np.load(f)
with open(f'./text_processing/useful_processed_text/useful_{model}_val.npy', 'rb') as f:
    val_array = np.load(f)
with open(f'./text_processing/useful_processed_text/useful_{model}_test.npy', 'rb') as f:
    test_array = np.load(f)

for i in [train_array, val_array, test_array]:
    print(i.shape)

(896945, 60)
(98119, 60)
(126864, 60)


In [ ]:
dt_estimator = tree.DecisionTreeClassifier()
dt_estimator.fit(train_array, train.useful_label)
y_train_useful = dt_estimator.predict(train_array)
y_val_useful = dt_estimator.predict(val_array)
y_test_useful = dt_estimator.predict(test_array)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  1.0
confusion matrix: 
[[594845      0]
 [     0 302100]]
accuracy score:  0.5953586970923063
confusion matrix: 
[[48723 24878]
 [14825  9693]]
accuracy score:  0.6022669945768697
confusion matrix: 
[[65621 34409]
 [16049 10785]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=10)
dt_estimator.fit(train_array, train.useful_label)
y_train_useful = dt_estimator.predict(train_array)
y_val_useful = dt_estimator.predict(val_array)
y_test_useful = dt_estimator.predict(test_array)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.675991281516704
confusion matrix: 
[[556015  38830]
 [251788  50312]]
accuracy score:  0.7245793373352766
confusion matrix: 
[[67180  6421]
 [20603  3915]]
accuracy score:  0.7508986000756716
confusion matrix: 
[[90862  9168]
 [22434  4400]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(min_samples_split=130)
dt_estimator.fit(train_array, train.useful_label)
y_train_useful = dt_estimator.predict(train_array)
y_val_useful = dt_estimator.predict(val_array)
y_test_useful = dt_estimator.predict(test_array)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.7456544158225978
confusion matrix: 
[[521583  73262]
 [154872 147228]]
accuracy score:  0.6504856347904076
confusion matrix: 
[[56054 17547]
 [16747  7771]]
accuracy score:  0.6637265102787236
confusion matrix: 
[[75685 24345]
 [18316  8518]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=10, min_samples_split=130)
dt_estimator.fit(train_array, train.useful_label)
y_train_useful = dt_estimator.predict(train_array)
y_val_useful = dt_estimator.predict(val_array)
y_test_useful = dt_estimator.predict(test_array)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.6755932638010135
confusion matrix: 
[[555691  39154]
 [251821  50279]]
accuracy score:  0.7246302958652249
confusion matrix: 
[[67170  6431]
 [20588  3930]]
accuracy score:  0.7507094211123723
confusion matrix: 
[[90828  9202]
 [22424  4410]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=7, min_samples_split=200)
dt_estimator.fit(train_array, train.useful_label)
y_train_useful = dt_estimator.predict(train_array)
y_val_useful = dt_estimator.predict(val_array)
y_test_useful = dt_estimator.predict(test_array)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.6668190357268283
confusion matrix: 
[[581665  13180]
 [285665  16435]]
accuracy score:  0.7434441851221476
confusion matrix: 
[[71548  2053]
 [23120  1398]]
accuracy score:  0.7756810442678774
confusion matrix: 
[[96985  3045]
 [25413  1421]]


In [ ]:
dt_estimator = ensemble.GradientBoostingClassifier(learning_rate=0.1)
dt_estimator.fit(train_array, train.useful_label)
y_train_useful = dt_estimator.predict(train_array)
y_val_useful = dt_estimator.predict(val_array)
y_test_useful = dt_estimator.predict(test_array)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.6752119695187553
confusion matrix: 
[[576421  18424]
 [272893  29207]]
accuracy score:  0.7453602258482047
confusion matrix: 
[[70478  3123]
 [21862  2656]]
accuracy score:  0.774498675747257
confusion matrix: 
[[95236  4794]
 [23814  3020]]


In [ ]:
dt_estimator = ensemble.AdaBoostClassifier(learning_rate=0.1)
dt_estimator.fit(train_array, train.useful_label)
y_train_useful = dt_estimator.predict(train_array)
y_val_useful = dt_estimator.predict(val_array)
y_test_useful = dt_estimator.predict(test_array)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.6631922804631276
confusion matrix: 
[[594844      1]
 [302097      3]]
accuracy score:  0.7501197525453785
confusion matrix: 
[[73601     0]
 [24518     0]]
accuracy score:  0.7884979190314038
confusion matrix: 
[[100030      0]
 [ 26832      2]]


In [ ]:
dt_estimator = XGBClassifier()
dt_estimator.fit(train_array, train.useful_label)
y_train_useful = dt_estimator.predict(train_array)
y_val_useful = dt_estimator.predict(val_array)
y_test_useful = dt_estimator.predict(test_array)
print("accuracy score: ", accuracy_score(train.useful_label, y_train_useful))
print("confusion matrix: ")
print(confusion_matrix(train.useful_label, y_train_useful))
print("accuracy score: ", accuracy_score(val.useful_label, y_val_useful))
print("confusion matrix: ")
print(confusion_matrix(val.useful_label, y_val_useful))
print("accuracy score: ", accuracy_score(test.useful_label, y_test_useful))
print("confusion matrix: ")
print(confusion_matrix(test.useful_label, y_test_useful))

accuracy score:  0.6749856457196373
confusion matrix: 
[[576994  17851]
 [273669  28431]]
accuracy score:  0.745350034142215
confusion matrix: 
[[70539  3062]
 [21924  2594]]
accuracy score:  0.7750346828099381
confusion matrix: 
[[95368  4662]
 [23878  2956]]


# funny

model = 'doc2vec'

In [ ]:
model = 'doc2vec'
with open(f'./text_processing/funny_processed_text/funny_{model}_train.npy', 'rb') as f:
    train_array = np.load(f)
with open(f'./text_processing/funny_processed_text/funny_{model}_val.npy', 'rb') as f:
    val_array = np.load(f)
with open(f'./text_processing/funny_processed_text/funny_{model}_test.npy', 'rb') as f:
    test_array = np.load(f)

for i in [train_array_fun, val_array_fun, test_array_fun]:
    print(i.shape)

(294305, 60)
(98119, 60)
(126864, 60)


In [ ]:
dt_estimator = tree.DecisionTreeClassifier()
dt_estimator.fit(train_array_fun, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun)
y_val_funny = dt_estimator.predict(val_array_fun)
y_test_funny = dt_estimator.predict(test_array_fun)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  1.0
confusion matrix: 
[[176583      0]
 [     0 117722]]
accuracy score:  0.5759842640059519
confusion matrix: 
[[52940 36216]
 [ 5388  3575]]
accuracy score:  0.5777289065455922
confusion matrix: 
[[68972 47464]
 [ 6107  4321]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=10)
dt_estimator.fit(train_array_fun, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun)
y_val_funny = dt_estimator.predict(val_array_fun)
y_test_funny = dt_estimator.predict(test_array_fun)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6035065663172559
confusion matrix: 
[[175971    612]
 [116078   1644]]
accuracy score:  0.9034335857479183
confusion matrix: 
[[88564   592]
 [ 8883    80]]
accuracy score:  0.9118741329297515
confusion matrix: 
[[115567    869]
 [ 10311    117]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=15)
dt_estimator.fit(train_array_fun, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun)
y_val_funny = dt_estimator.predict(val_array_fun)
y_test_funny = dt_estimator.predict(test_array_fun)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6150660029561169
confusion matrix: 
[[174092   2491]
 [110797   6925]]
accuracy score:  0.8838043600118224
confusion matrix: 
[[86421  2735]
 [ 8666   297]]
accuracy score:  0.8918684575608525
confusion matrix: 
[[112809   3627]
 [ 10091    337]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=7, min_samples_split=50)
dt_estimator.fit(train_array_fun, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun)
y_val_funny = dt_estimator.predict(val_array_fun)
y_test_funny = dt_estimator.predict(test_array_fun)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6010261463447784
confusion matrix: 
[[176145    438]
 [116982    740]]
accuracy score:  0.9062057297771073
confusion matrix: 
[[88872   284]
 [ 8919    44]]
accuracy score:  0.9149009963425401
confusion matrix: 
[[116013    423]
 [ 10373     55]]


In [ ]:
dt_estimator = ensemble.GradientBoostingClassifier(learning_rate=0.1)
dt_estimator.fit(train_array_fun, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun)
y_val_funny = dt_estimator.predict(val_array_fun)
y_test_funny = dt_estimator.predict(test_array_fun)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6005368580214403
confusion matrix: 
[[176573     10]
 [117554    168]]
accuracy score:  0.9083459880349372
confusion matrix: 
[[89121    35]
 [ 8958     5]]
accuracy score:  0.91732091058141
confusion matrix: 
[[116369     67]
 [ 10422      6]]


In [ ]:
dt_estimator = ensemble.AdaBoostClassifier(learning_rate=0.1)
dt_estimator.fit(train_array_fun, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun)
y_val_funny = dt_estimator.predict(val_array_fun)
y_test_funny = dt_estimator.predict(test_array_fun)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6
confusion matrix: 
[[176583      0]
 [117722      0]]
accuracy score:  0.9086517392146272
confusion matrix: 
[[89156     0]
 [ 8963     0]]
accuracy score:  0.9178017404464623
confusion matrix: 
[[116436      0]
 [ 10428      0]]


In [ ]:
dt_estimator = XGBClassifier()
dt_estimator.fit(train_array_fun, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun)
y_val_funny = dt_estimator.predict(val_array_fun)
y_test_funny = dt_estimator.predict(test_array_fun)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6003227943799799
confusion matrix: 
[[176577      6]
 [117621    101]]
accuracy score:  0.9084580968008235
confusion matrix: 
[[89134    22]
 [ 8960     3]]
accuracy score:  0.9175652667423383
confusion matrix: 
[[116401     35]
 [ 10423      5]]


model = 'sbert'

In [ ]:
model = 'sbert'
with open(f'./text_processing/funny_processed_text/funny_{model}_train.npy', 'rb') as f:
    train_array = np.load(f)
with open(f'./text_processing/funny_processed_text/funny_{model}_val.npy', 'rb') as f:
    val_array = np.load(f)
with open(f'./text_processing/funny_processed_text/funny_{model}_test.npy', 'rb') as f:
    test_array = np.load(f)

for i in [train_array_fun, val_array_fun, test_array_fun]:
    print(i.shape)

(294305, 768)
(98119, 768)
(126864, 768)


In [ ]:
dt_estimator = tree.DecisionTreeClassifier()
dt_estimator.fit(train_array_fun, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun)
y_val_funny = dt_estimator.predict(val_array_fun)
y_test_funny = dt_estimator.predict(test_array_fun)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.9999150541105316
confusion matrix: 
[[176583      0]
 [    25 117697]]
accuracy score:  0.5730796277988972
confusion matrix: 
[[52625 36531]
 [ 5358  3605]]
accuracy score:  0.5756085256652793
confusion matrix: 
[[68881 47555]
 [ 6285  4143]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=30)
dt_estimator.fit(train_array_fun, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun)
y_val_funny = dt_estimator.predict(val_array_fun)
y_test_funny = dt_estimator.predict(test_array_fun)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6966242503525254
confusion matrix: 
[[174005   2578]
 [ 86707  31015]]
accuracy score:  0.8119426410786902
confusion matrix: 
[[78692 10464]
 [ 7988   975]]
accuracy score:  0.8185694917391853
confusion matrix: 
[[102793  13643]
 [  9374   1054]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=15)
dt_estimator.fit(train_array_fun, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun)
y_val_funny = dt_estimator.predict(val_array_fun)
y_test_funny = dt_estimator.predict(test_array_fun)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6144441990452082
confusion matrix: 
[[175521   1062]
 [112409   5313]]
accuracy score:  0.8889715549485828
confusion matrix: 
[[87083  2073]
 [ 8821   142]]
accuracy score:  0.8982690124858116
confusion matrix: 
[[113797   2639]
 [ 10267    161]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=7, min_samples_split=50)
dt_estimator.fit(train_array_fun, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun)
y_val_funny = dt_estimator.predict(val_array_fun)
y_test_funny = dt_estimator.predict(test_array_fun)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6015765957085336
confusion matrix: 
[[176091    492]
 [116766    956]]
accuracy score:  0.9036679949856806
confusion matrix: 
[[88635   521]
 [ 8931    32]]
accuracy score:  0.912993441795939
confusion matrix: 
[[115783    653]
 [ 10385     43]]


In [ ]:
dt_estimator = ensemble.GradientBoostingClassifier(learning_rate=0.1)
dt_estimator.fit(train_array_fun, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun)
y_val_funny = dt_estimator.predict(val_array_fun)
y_test_funny = dt_estimator.predict(test_array_fun)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6007441259917432
confusion matrix: 
[[176580      3]
 [117500    222]]
accuracy score:  0.9080911953851956
confusion matrix: 
[[89100    56]
 [ 8962     1]]
accuracy score:  0.9172184386429563
confusion matrix: 
[[116359     77]
 [ 10425      3]]


In [ ]:
dt_estimator = XGBClassifier()
dt_estimator.fit(train_array_fun, train_fun.funny_label)
y_train_funny = dt_estimator.predict(train_array_fun)
y_val_funny = dt_estimator.predict(val_array_fun)
y_test_funny = dt_estimator.predict(test_array_fun)
print("accuracy score: ", accuracy_score(train_fun.funny_label, y_train_funny))
print("confusion matrix: ")
print(confusion_matrix(train_fun.funny_label, y_train_funny))
print("accuracy score: ", accuracy_score(val.funny_label, y_val_funny))
print("confusion matrix: ")
print(confusion_matrix(val.funny_label, y_val_funny))
print("accuracy score: ", accuracy_score(test.funny_label, y_test_funny))
print("confusion matrix: ")
print(confusion_matrix(test.funny_label, y_test_funny))

accuracy score:  0.6003363857222949
confusion matrix: 
[[176581      2]
 [117621    101]]
accuracy score:  0.9083561797409269
confusion matrix: 
[[89127    29]
 [ 8963     0]]
accuracy score:  0.9175179720015134
confusion matrix: 
[[116399     37]
 [ 10427      1]]


# cool

In [ ]:
model = 'doc2vec'
with open(f'./text_processing/cool_processed_text/cool_{model}_train.npy', 'rb') as f:
    train_array = np.load(f)
with open(f'./text_processing/cool_processed_text/cool_{model}_val.npy', 'rb') as f:
    val_array = np.load(f)
with open(f'./text_processing/cool_processed_text/cool_{model}_test.npy', 'rb') as f:
    test_array = np.load(f)

for i in [train_array_cool, val_array_cool, test_array_cool]:
    print(i.shape)

(376157, 60)
(98119, 60)
(126864, 60)


In [ ]:
dt_estimator = tree.DecisionTreeClassifier()
dt_estimator.fit(train_array_cool, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool)
y_val_cool = dt_estimator.predict(val_array_cool)
y_test_cool = dt_estimator.predict(test_array_cool)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  1.0
confusion matrix: 
[[225694      0]
 [     0 150463]]
accuracy score:  0.5638153670542912
confusion matrix: 
[[49685 34614]
 [ 8184  5636]]
accuracy score:  0.5692631479379493
confusion matrix: 
[[65519 44876]
 [ 9769  6700]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=20)
dt_estimator.fit(train_array_cool, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool)
y_val_cool = dt_estimator.predict(val_array_cool)
y_test_cool = dt_estimator.predict(test_array_cool)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  0.6205068628258945
confusion matrix: 
[[223124   2570]
 [140179  10284]]
accuracy score:  0.8333350319509982
confusion matrix: 
[[81296  3003]
 [13350   470]]
accuracy score:  0.8433677008449993
confusion matrix: 
[[106407   3988]
 [ 15883    586]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=30)
dt_estimator.fit(train_array_cool, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool)
y_val_cool = dt_estimator.predict(val_array_cool)
y_test_cool = dt_estimator.predict(test_array_cool)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  0.6850915973915147
confusion matrix: 
[[221114   4580]
 [113875  36588]]
accuracy score:  0.7745288883906277
confusion matrix: 
[[74509  9790]
 [12333  1487]]
accuracy score:  0.7830669062933535
confusion matrix: 
[[97600 12795]
 [14726  1743]]


In [ ]:
dt_estimator = tree.DecisionTreeClassifier(max_depth=7, min_samples_split=50)
dt_estimator.fit(train_array_cool, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool)
y_val_cool = dt_estimator.predict(val_array_cool)
y_test_cool = dt_estimator.predict(test_array_cool)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  0.6005338196550908
confusion matrix: 
[[224992    702]
 [149560    903]]
accuracy score:  0.8562663704277459
confusion matrix: 
[[83964   335]
 [13768    52]]
accuracy score:  0.8672357800479253
confusion matrix: 
[[109951    444]
 [ 16399     70]]


In [ ]:
dt_estimator = ensemble.GradientBoostingClassifier(learning_rate=0.1)
dt_estimator.fit(train_array_cool, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool)
y_val_cool = dt_estimator.predict(val_array_cool)
y_test_cool = dt_estimator.predict(test_array_cool)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  0.6004886257599885
confusion matrix: 
[[225687      7]
 [150272    191]]
accuracy score:  0.8588958305730796
confusion matrix: 
[[84272    27]
 [13818     2]]
accuracy score:  0.8699236978181359
confusion matrix: 
[[110353     42]
 [ 16460      9]]


In [ ]:
dt_estimator = ensemble.AdaBoostClassifier(learning_rate=0.1)
dt_estimator.fit(train_array_cool, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool)
y_val_cool = dt_estimator.predict(val_array_cool)
y_test_cool = dt_estimator.predict(test_array_cool)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  0.6000180775580409
confusion matrix: 
[[225694      0]
 [150456      7]]
accuracy score:  0.8591506232228213
confusion matrix: 
[[84299     0]
 [13820     0]]
accuracy score:  0.8701444066086518
confusion matrix: 
[[110389      6]
 [ 16468      1]]


In [ ]:
dt_estimator = XGBClassifier()
dt_estimator.fit(train_array_cool, train_cool.cool_label)
y_train_cool = dt_estimator.predict(train_array_cool)
y_val_cool = dt_estimator.predict(val_array_cool)
y_test_cool = dt_estimator.predict(test_array_cool)
print("accuracy score: ", accuracy_score(train_cool.cool_label, y_train_cool))
print("confusion matrix: ")
print(confusion_matrix(train_cool.cool_label, y_train_cool))
print("accuracy score: ", accuracy_score(val.cool_label, y_val_cool))
print("confusion matrix: ")
print(confusion_matrix(val.cool_label, y_val_cool))
print("accuracy score: ", accuracy_score(test.cool_label, y_test_cool))
print("confusion matrix: ")
print(confusion_matrix(test.cool_label, y_test_cool))

accuracy score:  0.6002839239998192
confusion matrix: 
[[225686      8]
 [150348    115]]
accuracy score:  0.8589671725150073
confusion matrix: 
[[84279    20]
 [13818     2]]
accuracy score:  0.8700025223861774
confusion matrix: 
[[110367     28]
 [ 16464      5]]
